This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [31]:
# First, import the relevant modules
import requests as re
import datetime as dt

In [11]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://www.quandl.com/api/v3/datasets/{database_code}/{dataset_code}/data.{return_format}'.format(
    database_code = 'FSE',
    dataset_code  = 'AFX_X',
    return_format = 'json'
)
response = re.get(
    url,
    params = {
        'start_date': '2019-11-12',
        'end_date': '2019-11-12'
    }
)

In [13]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
response.json()

{'dataset_data': {'limit': None,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2019-11-12',
  'end_date': '2019-11-12',
  'frequency': 'daily',
  'data': [['2019-11-12',
    101.1,
    102.0,
    98.9,
    101.0,
    None,
    128193.0,
    12877095.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

---

##### Part 1: Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [17]:
re_data = re.get(
    url,
    params = {
        'start_date': '2017-01-01',
        'end_date': '2017-12-31'
    }
)

##### Part 2: Convert the returned JSON object into a Python dictionary.
* Create a dict to store our data `dict`
* Extract the label information and remove the date from this list
* Extract the date element from each data element
* Store the data for each date (key) in our data object, stored as `dict`s

In [42]:
data = {}
labels = re_data.json()['dataset_data']['column_names']
labels.remove('Date')

for date_data in re_data.json()['dataset_data']['data']:
    # extract the date, but convert to a date object
    date = dt.datetime.strptime(date_data[0], '%Y-%m-%d').date()
    data[date] = dict(zip(labels, date_data[1:]))

##### Part 3: Calculate what the highest and lowest opening prices were for the stock in this period.
* Note that there exist values of None in the dataset
    * useing the `if data_elements['Open']` in the list comprehension eliminates these `None` values

In [75]:
min_opening_value = min([data_elements['Open'] for data_elements in data.values() if data_elements['Open']])
min_opening_value

34.0

In [74]:
max_opening_value = max([data_elements['Open'] for data_elements in data.values() if data_elements['Open']])
max_opening_value

53.11

##### Part 4: What was the largest change in any one day (based on High and Low price)?

In [73]:
max_daily_delta = max([data_elements['High'] - data_elements['Low'] for data_elements in data.values() \
     if (data_elements['High']) if (data_elements['Low'])])
max_daily_delta

2.8100000000000023

##### Part 5: What was the largest change between any two days (based on Closing Price)?
* Compare the closing price of day x with day x+1

In [72]:
max_close_change_delta = 0

for date, value_set_1 in data.items():
    value_set_2 = data.get(date + dt.timedelta(days=1))
    
    if value_set_2 is not None:
        delta = abs(value_set_1['Close'] - value_set_2['Close'])
        
        if delta > max_close_change_delta:
            max_close_change_delta = delta

max_close_change_delta

2.559999999999995

##### Part 6: What was the average daily trading volume during this year?

In [84]:
d = [data_elements['Traded Volume'] for data_elements in data.values()]
avg_trade_volume = sum(d)/len(d)
avg_trade_volume

89124.33725490196

##### Part 7: (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)
* sort and find the middle value
    * note that if there are two middle values, we find the average of these.

In [106]:
d.sort()
l = len(d)
if (l%2) == 1:
    i = int((l-1)/2)
    median_trade_volume = sum(d[i:i+1])/2
else:
    i = int(l/2)
    median_trade_volume = d[i]

median_trade_volume

38143.0